In [3]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import GridSearchCV
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np
from tqdm import tqdm

In [4]:
def bootstrap_resample(data, num_iterations):
    resampled_models = []
    n = len(data)
    
    for _ in range(num_iterations):
        # Generate a bootstrap sample by randomly sampling with replacement
        bootstrap_sample = data.sample(n=n, replace=True)
        
        # Append the bootstrap sample to the list of resampled models
        resampled_models.append(bootstrap_sample)
    
    return resampled_models

In [5]:
train_df = pd.read_excel('train_data_19_03.xlsx', index_col=False)

In [6]:
test_df = pd.read_excel('test_data_19_03.xlsx', index_col=False)

In [7]:
num_resampled_models = 500

# Perform bootstrap resampling
resampled_models = bootstrap_resample(train_df, num_resampled_models)

# Print the first resampled model (as an example)
print("Example of a resampled model:")
print(resampled_models[0].info())

Example of a resampled model:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 3969 entries, 3161 to 312
Data columns (total 36 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Hm0 d     3969 non-null   float64
 1   Tp d      3969 non-null   float64
 2   Tm d      3969 non-null   float64
 3   Tm1,0 d   3969 non-null   float64
 4   hdeep     3969 non-null   float64
 5   mmm       3969 non-null   float64
 6   β         3969 non-null   float64
 7   Spread s  3969 non-null   float64
 8   h         3969 non-null   float64
 9   Hm0 toe   3969 non-null   float64
 10  Tp toe    3969 non-null   float64
 11  Tm toe    3969 non-null   float64
 12  Tm1,0t    3969 non-null   float64
 13  ht        3969 non-null   float64
 14  Bt        3969 non-null   float64
 15  cotαd     3969 non-null   float64
 16  cotαu     3969 non-null   float64
 17  cotαexcl  3969 non-null   float64
 18  cotαincl  3969 non-null   float64
 19  gf_d      3969 non-null   float64
 20

In [8]:
len(resampled_models)

500

In [11]:
X_test = test_df[test_df.columns[:-1]]
y_test = test_df[test_df.columns[-1]]
all_rmse = []
all_mape = []
all_preds = []

for dataset_sample in tqdm(resampled_models):
    regressor = xgb.XGBRegressor()
    
    xg_reg = xgb.XGBRegressor(
        objective ='reg:squarederror', 
        learning_rate = 0.07,
        max_depth = 21,
        min_child_weight=3,
        reg_lambda=1,
        subsample=0.75
    )
    
    X_train = dataset_sample[dataset_sample.columns[:-1]]
    y_train = dataset_sample[dataset_sample.columns[-1]]
    xg_reg.fit(X_train, y_train)
    
    preds = xg_reg.predict(X_test)
    
    all_rmse.append(np.sqrt(mean_squared_error(y_test, preds)))
    all_mape.append(mean_absolute_percentage_error(y_test, preds))
    all_preds.append(preds)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [28:39<00:00,  3.44s/it]


In [16]:
eval_df = pd.DataFrame({
    'rmse': all_rmse,
    'mape': all_mape,
    'preds': all_preds
}) 

In [17]:
eval_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   rmse    500 non-null    float64
 1   mape    500 non-null    float64
 2   preds   500 non-null    object 
dtypes: float64(2), object(1)
memory usage: 11.8+ KB


In [18]:
eval_df.head()

,rmse,mape,preds
0,0.330631,0.057263,"[-3.922778, -3.9782982, -2.9749436, -3.2239969..."
1,0.316845,0.056041,"[-4.084008, -4.3743, -2.900655, -3.127904, -4...."
2,0.325540,0.056362,"[-4.016885, -4.322522, -2.9363346, -3.209578, ..."
3,0.332213,0.058454,"[-3.977727, -4.3450236, -3.245346, -3.2484121,..."
4,0.323408,0.055971,"[-3.953535, -4.0700684, -3.1048672, -3.2616057..."


In [19]:
all_bias = []
all_R_sq = []
all_r = []
for preds in all_preds:
    all_bias.append((preds-y_test).sum()/len(preds))
    all_R_sq.append(sklearn.metrics.r2_score(y_test, preds))
    all_r.append(np.corrcoef(y_test, preds)[0, 1])

In [20]:
eval_df = pd.DataFrame({
    'rmse': all_rmse,
    'mape': all_mape,
    'preds': all_preds,
    'bias': all_bias,
    'R_sq': all_R_sq,
    'r': all_r
}) 

In [21]:
eval_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   rmse    500 non-null    float64
 1   mape    500 non-null    float64
 2   preds   500 non-null    object 
 3   bias    500 non-null    float64
 4   R_sq    500 non-null    float64
 5   r       500 non-null    float64
dtypes: float64(5), object(1)
memory usage: 23.6+ KB


In [22]:
eval_df.to_excel('bootstraping_results.xlsx')

In [26]:
preds_df = pd.DataFrame(all_preds).T

In [29]:
preds_df.shape

(1701, 500)

In [30]:
preds_df.to_excel('bootstraping_preds.xlsx')

In [31]:
mean_rmse = np.mean(all_rmse)

In [32]:
mean_rmse

0.3286887445994513

In [33]:
best_rmse = np.min(all_rmse)

In [34]:
best_rmse

0.3123423401904806

In [36]:
mean_mape = np.mean(all_mape)
best_mape = np.min(all_mape)
mean_mape, best_mape

(0.05746587873252982, 0.05437500518558259)

In [38]:
mean_r = np.mean(all_r)
best_r = np.max(all_r)
mean_r, best_r

(0.9414957660771202, 0.9473594466734909)